# Ingesting data regarding child pedestrians/cyclists involved in street accidents around the Indianapolis Metro area

In [2]:
using HTTP, JSON, DataFrames, JuliaDB, CSV, DataValues

The data is derived from the ARIES database of crash data is maintained by the Indiana State Police, they have published separate tables for each calendar year.

In [3]:
#Form an array with dataset ID's for each year

#=
The first six years do not return any results from the SQL query outlined one cell below:

d2007 = "994487c5-5d44-4fb9-ae9d-84ca75659bf0"
d2008 = "e4e56445-8cb6-4fc5-955c-7a2331684279"
d2009 = "881391be-d712-44b4-bfd0-ef9f018a4e4b"
d2010 = "46eb8985-f5b5-4635-9d6c-769dbd05fa02"
d2011 = "32ecfbfa-cf4b-4f64-af48-e1a624ae5ba4"
d2012 = "c2727405-925d-48fe-a8c1-06058ee65a25"
=#

d2013 = "ca9adbd5-001e-49e7-a850-f9f3471eb2db"
d2014 = "a5262209-109c-49cd-b6a9-a72970dc1cf5"
d2015 = "7fd801c4-4cf3-4e18-a949-a4d21401d798"
d2016 = "88646815-5b52-433b-9609-c75500e48e4c"
d2017 = "911880b1-d0bb-4168-a5c1-a52dcc291445"

dARIES = [d2013,d2014,d2015,d2016,d2017]

5-element Array{String,1}:
 "ca9adbd5-001e-49e7-a850-f9f3471eb2db"
 "a5262209-109c-49cd-b6a9-a72970dc1cf5"
 "7fd801c4-4cf3-4e18-a949-a4d21401d798"
 "88646815-5b52-433b-9609-c75500e48e4c"
 "911880b1-d0bb-4168-a5c1-a52dcc291445"

In [4]:
#For each dataset, send the URL-encoded SQL query below that filters by age, mode of travel, and location. Convert the response into JSON, then into a local dataframe, then append it onto the global dataframe.

#=
SELECT * from "[DATASET]"
    WHERE "PERSONTYPEDESCR" IN ('Pedal Cyclist','Pedestrian') 
    AND "AGE_GRP" IN ('1-4 Years','5-14 Years','15-24 Years')
    AND "COUNTDESCR" IN ('Marion','Boone','Hamilton','Hancock','Shelby','Johnson','Morgan','Hendricks')
=#

df = DataFrame()

for dataset in dARIES
    try
        address = string("https://hub.mph.in.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22",dataset,"%22%20WHERE%20%22PERSONTYPEDESCR%22%20IN%20(%27Pedal%20Cyclist%27,%27Pedestrian%27)%20AND%20%22AGE_GRP%22%20IN%20(%271-4%20Years%27,%275-14%20Years%27,%2715-24%20Years%27)%20AND%20%22COUNTYDESCR%22%20IN%20(%27Marion%27,%27Boone%27,%27Hamilton%27,%27Hancock%27,%27Shelby%27,%27Johnson%27,%27Morgan%27,%27Hendricks%27)")
        response = HTTP.get(address)
        rawData = response.body |> String |> JSON.parse
        records = rawData["result"]["records"]
        locdf = DataFrame()
        for key in keys(first(records))
            locdf[!,Symbol(key)] = [record[key] for record in records]
        end
        df = vcat(df,locdf)
    catch
        @warn "Error thrown, will skip to next dataset!"
    end
end

#Thanks to @rdeits on Discourse for helping write a loop that constructs the dataframe by column rather than by row

┌ Warning: Error thrown, will skip to next dataset!
└ @ Main In[4]:24


In [5]:
# Write to CSV

df |> CSV.write("compositeARIES.csv")

ArgumentError: ArgumentError: `nothing` should not be printed; use `show`, `repr`, or custom output instead.